<h1>Recommender system challenge PoliMi 2018</h1>

<b>Import dependencies</b>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import recommender as recommender

<b>See if we upload it correctly</b>

In [5]:
train = pd.read_csv("all/train.csv")
train.head()

,playlist_id,track_id
0,0,14301
1,0,8360
2,0,12844
3,0,18397
4,0,1220


In [6]:
playlist_list = list(np.asarray(train['playlist_id']))
track_list = list(np.asarray(train['track_id']))
playlist_list[0:10]

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [7]:
tracks = pd.read_csv("all/tracks.csv")
tracks.head()

,track_id,album_id,artist_id,duration_sec
0,0,6306,449,167
1,1,12085,4903,185
2,2,1885,6358,201
3,3,3989,1150,263
4,4,11633,4447,96


In [8]:
test = pd.read_csv("all/target_playlists.csv")
test.head()

,playlist_id
0,7
1,25
2,29
3,34
4,50


In [9]:
test_playlist_list = list(np.asarray(test['playlist_id']))
all_track_list = list(np.asarray(train['track_id']))
album_list = list(np.asarray(tracks['album_id']))
artist_list = list(np.asarray(tracks['artist_id']))
duration_list = list(np.asarray(tracks['duration_sec']))
all_tracks_in_tracks_list = list(np.asarray(tracks['track_id']))


In [10]:
#number of different playlist, tracks, albums, artists
playlist_unique = list(set(playlist_list))
track_unique = list(set(track_list))
album_unique = list(set(album_list))
artist_unique = list(set(artist_list))

num_playlists = len(playlist_list)
num_tracks = len(track_list)
num_albums = len(album_unique)
num_artists = len(artist_unique)

num_tracks

1211791

In [11]:
data = np.ones((num_playlists), dtype=int)

In [12]:
import scipy.sparse as sps
                         
URM_train = sps.coo_matrix((data, (playlist_list, track_list)))

URM_train

<50446x20635 sparse matrix of type '<class 'numpy.int64'>'
	with 1211791 stored elements in COOrdinate format>

In [13]:
URM_train.tocsr()

<50446x20635 sparse matrix of type '<class 'numpy.int64'>'
	with 1211791 stored elements in Compressed Sparse Row format>

In [14]:
train_test_split = 0.80

numInteractions = URM_train.getnnz()
numInteractions

1211791

In [15]:
train_mask = np.random.choice([True,False], numInteractions, p=[train_test_split, 1-train_test_split])
train_mask

array([ True, False,  True, ...,  True,  True, False])

In [16]:
track_list = np.array(track_list)
playlist_list = np.array(playlist_list)
all_track_list = np.array(all_track_list)
#train_dummies = np.array(train_dummies)

#len(train_dummies)
#len(train_mask)

URM_train = sps.coo_matrix((data[train_mask], (playlist_list[train_mask], track_list[train_mask])))
URM_train = URM_train.tocsr()
URM_train

<50446x20635 sparse matrix of type '<class 'numpy.int64'>'
	with 969579 stored elements in Compressed Sparse Row format>

In [17]:
test_mask = np.logical_not(train_mask)

URM_test = sps.coo_matrix((data[test_mask], (playlist_list[test_mask], track_list[test_mask])))
URM_test = URM_test.tocsr()
URM_test

<50446x20635 sparse matrix of type '<class 'numpy.int64'>'
	with 242212 stored elements in Compressed Sparse Row format>

<h3>Mean Average Precision</h3>

In [18]:
def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [19]:
def evaluate_algorithm_different(URM_test, recommender_object, at=10):
    
    cumulative_MAP = 0.0
    
    num_eval = 0

    for playlist_id in playlist_unique:

        relevant_items = URM_test[playlist_id].indices
        
        if len(relevant_items)>0:
            #recommended_items = recommender_object.recommend(playlist_id, at=at)
            recommended_items = recommender_object.recommend(playlist_id)
            num_eval+=1
            cumulative_MAP += MAP(recommended_items, relevant_items)


    cumulative_MAP /= num_eval
    
    print("Recommender performance is: MAP = {:.4f}".format(cumulative_MAP)) 

In [20]:
def evaluate_algorithm(URM_test, recommender_object, at=10):
    
    cumulative_MAP = 0.0
    
    num_eval = 0

    for playlist_id in playlist_unique:

        relevant_items = URM_test[playlist_id].indices
        
        if len(relevant_items)>0:
            recommended_items = recommender_object.recommend(playlist_id, at=at)
            num_eval+=1
            cumulative_MAP += MAP(recommended_items, relevant_items)


    cumulative_MAP /= num_eval
    
    print("Recommender performance is: MAP = {:.4f}".format(cumulative_MAP)) 

<h2>TopPop Recommender</h2>

In [39]:
class TopPopRecommender(object):

    def fit(self, URM_train):
    
        self.URM_train = URM_train
    
        itemPopularity = (URM_train>0).sum(axis=0)
        itemPopularity = np.array(itemPopularity).squeeze()

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popularItems = np.argsort(itemPopularity)
        self.popularItems = np.flip(self.popularItems, axis = 0)
    
    
    def recommend(self, playlist_id, at=10, remove_seen=True):
        
        if remove_seen:
            unseen_items_mask = np.in1d(self.popularItems, 
                                        self.URM_train[playlist_id].indices, 
                                        assume_unique = True, 
                                        invert = True)
            
            unseen_items = self.popularItems[unseen_items_mask]

            recommended_items = unseen_items[0:at]
        
        else:
            recommended_items = self.popularItems[0:at]

        return recommended_items

<h3>Fit and test the model</h3>

In [70]:
topPopRecommender = TopPopRecommender()
topPopRecommender.fit(URM_train)

In [41]:
for playlist_id in playlist_unique[0:10]:
    print(topPopRecommender.recommend(playlist_id, at=10))

[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]
[ 8956 10848  5606 15578 10496  2674 13980 17239 18266  2272]


In [21]:
evaluate_algorithm(URM_test, topPopRecommender, at=10)

Recommender performance is: MAP = 0.0043


<h2>Works!</h2>

<h3>Code for creation of the submission csv</h3>

In [131]:
test_playlist_list.sort()

In [132]:
submission = pd.DataFrame(columns=["playlist_id","track_ids"])

for playlist_id in test_playlist_list: 
    recommendation = ' '.join(map(str, topPopRecommender.recommend(playlist_id, at=10)))
    row = pd.DataFrame([[playlist_id,recommendation]], columns=["playlist_id","track_ids"])
    submission = submission.append(row)

submission.to_csv("all/sub.csv", index = False)

<h2>Content based similarity</h2>

Build ICM

In [19]:
ones = np.ones((num_tracks_in_tracks), dtype=int)

ICM_all_artist = sps.coo_matrix((ones, (tracks_in_tracks, artist_list)))
ICM_all_artist = ICM_all_artist.tocsr()

ICM_all_artist

<20635x6668 sparse matrix of type '<class 'numpy.int64'>'
	with 20635 stored elements in Compressed Sparse Row format>

In [20]:
ones = np.ones((num_tracks_in_tracks), dtype=int)

ICM_all_album = sps.coo_matrix((ones, (tracks_in_tracks, album_list)))
ICM_all_album = ICM_all_album.tocsr()

ICM_all_album

<20635x12744 sparse matrix of type '<class 'numpy.int64'>'
	with 20635 stored elements in Compressed Sparse Row format>

In [21]:
ones = np.ones((num_tracks_in_tracks), dtype=int)

ICM_all_duration = sps.coo_matrix((ones, (tracks_in_tracks, duration_list)))
ICM_all_duration = ICM_all_duration.tocsr()

ICM_all_duration

<20635x2115 sparse matrix of type '<class 'numpy.int64'>'
	with 20635 stored elements in Compressed Sparse Row format>

In [22]:
class BasicItemKNNRecommender(object):
    """ ItemKNN recommender with cosine similarity and no shrinkage"""

    def __init__(self, URM, k=50, shrinkage=100, similarity='cosine'):
        self.dataset = URM
        self.k = k
        self.shrinkage = shrinkage
        self.similarity_name = similarity
        if similarity == 'cosine':
            self.distance = Cosine(shrinkage=self.shrinkage)
        elif similarity == 'pearson':
            self.distance = Pearson(shrinkage=self.shrinkage)
        elif similarity == 'adj-cosine':
            self.distance = AdjustedCosine(shrinkage=self.shrinkage)
        else:
            raise NotImplementedError('Distance {} not implemented'.format(similarity))

    def __str__(self):
        return "ItemKNN(similarity={},k={},shrinkage={})".format(
            self.similarity_name, self.k, self.shrinkage)

    def fit(self, X):
        item_weights = self.distance.compute(X)
        
        item_weights = check_matrix(item_weights, 'csr') # nearly 10 times faster
        print("Converted to csr")
        
        # for each column, keep only the top-k scored items
        # THIS IS THE SLOW PART, FIND A BETTER SOLUTION        
        values, rows, cols = [], [], []
        nitems = self.dataset.shape[1]
        for i in range(nitems):
            if (i % 10000 == 0):
                print("Item %d of %d" % (i, nitems))
                
            this_item_weights = item_weights[i,:].toarray()[0]
            top_k_idx = np.argsort(this_item_weights) [-self.k:]
                        
            values.extend(this_item_weights[top_k_idx])
            rows.extend(np.arange(nitems)[top_k_idx])
            cols.extend(np.ones(self.k) * i)
        self.W_sparse = sps.csc_matrix((values, (rows, cols)), shape=(nitems, nitems), dtype=np.float32)

    def recommend(self, playlist_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        user_profile = self.dataset[playlist_id]
        scores = user_profile.dot(self.W_sparse).toarray().ravel()

        # rank items
        ranking = scores.argsort()[::-1]
        if exclude_seen:
            ranking = self._filter_seen(playlist_id, ranking)
            
        return ranking[:at]
    
    def _filter_seen(self, playlist_id, ranking):
        user_profile = self.dataset[playlist_id]
        seen = user_profile.indices
        unseen_mask = np.in1d(ranking, seen, assume_unique=True, invert=True)
        return ranking[unseen_mask]

In [23]:
def check_matrix(X, format='csc', dtype=np.float32):
    if format == 'csc' and not isinstance(X, sps.csc_matrix):
        return X.tocsc().astype(dtype)
    elif format == 'csr' and not isinstance(X, sps.csr_matrix):
        return X.tocsr().astype(dtype)
    elif format == 'coo' and not isinstance(X, sps.coo_matrix):
        return X.tocoo().astype(dtype)
    elif format == 'dok' and not isinstance(X, sps.dok_matrix):
        return X.todok().astype(dtype)
    elif format == 'bsr' and not isinstance(X, sps.bsr_matrix):
        return X.tobsr().astype(dtype)
    elif format == 'dia' and not isinstance(X, sps.dia_matrix):
        return X.todia().astype(dtype)
    elif format == 'lil' and not isinstance(X, sps.lil_matrix):
        return X.tolil().astype(dtype)
    else:
        return X.astype(dtype)

In [21]:
import scipy
class ISimilarity(object):
    """Abstract interface for the similarity metrics"""

    def __init__(self, shrinkage=10):
        self.shrinkage = shrinkage

    def compute(self, X):
        pass


class Cosine(ISimilarity):
    def compute(self, X):
        # convert to csc matrix for faster column-wise operations
        X = check_matrix(X, 'csc', dtype=np.float32)

        # 1) normalize the columns in X
        # compute the column-wise norm
        # NOTE: this is slightly inefficient. We must copy X to compute the column norms.
        # A faster solution is to  normalize the matrix inplace with a Cython function.
        Xsq = X.copy()
        Xsq.data **= 2
        norm = np.sqrt(Xsq.sum(axis=0))
        norm = np.asarray(norm).ravel()
        norm += 1e-6
        # compute the number of non-zeros in each column
        # NOTE: this works only if X is instance of sparse.csc_matrix
        col_nnz = np.diff(X.indptr)
        # then normalize the values in each column
        X.data /= np.repeat(norm, col_nnz)
        print("Normalized")

        # 2) compute the cosine similarity using the dot-product
        dist = X * X.T
        print("Computed")
        
        # zero out diagonal values
        dist = dist - sps.dia_matrix((dist.diagonal()[scipy.newaxis, :], [0]), shape=dist.shape)
        print("Removed diagonal")
        
        # and apply the shrinkage
        if self.shrinkage > 0:
            dist = self.apply_shrinkage(X, dist)
            print("Applied shrinkage")    
        
        return dist

    def apply_shrinkage(self, X, dist):
        # create an "indicator" version of X (i.e. replace values in X with ones)
        X_ind = X.copy()
        X_ind.data = np.ones_like(X_ind.data)
        # compute the co-rated counts
        co_counts = X_ind * X_ind.T
        # remove the diagonal
        co_counts = co_counts - sps.dia_matrix((co_counts.diagonal()[scipy.newaxis, :], [0]), shape=co_counts.shape)
        # compute the shrinkage factor as co_counts_ij / (co_counts_ij + shrinkage)
        # then multiply dist with it
        co_counts_shrink = co_counts.copy()
        co_counts_shrink.data += self.shrinkage
        co_counts.data /= co_counts_shrink.data
        dist.data *= co_counts.data
        return dist


<h2>Test it</h2>

<h3>Artists</h3>

In [26]:
rec = BasicItemKNNRecommender(URM=URM_train, shrinkage=0.0, k=50)
rec.fit(ICM_all_artist)

Normalized
Computed
Removed diagonal
Converted to csr
Item 0 of 20635
Item 10000 of 20635
Item 20000 of 20635


In [27]:
for playlist_id in playlist_unique[0:10]:
    print(rec.recommend(playlist_id, at=10))

[ 6095 19582 10912 19034 18427  2355 14796  5957  1568  2506]
[11605  8907 16515 17519 14741 11861 18466  3896  8727 12388]
[15130 15679 11228 10472 15506 13364  3151  7112 12460  8796]
[ 5218  9348 13503  4379 11327  9740  3292 17348 19475  3321]
[ 9576  3493 10927 13052  2528 13812  3747 12601 10023 11958]
[  243 19620  8804  6879  6874  6875  6876  6877  6878  6886]
[ 4103  7906 13979   102  9550  2385  6954 10945  6725  7803]
[ 4542  9570  4032  3052  1651 11776  1790 15908  4492 15124]
[ 2672 14049 18630 15614  5179  6460 10585  1025 11251 17148]
[ 3523  2230 18769  7031 11277  7646 14730  3200 20513  3659]


In [28]:
evaluate_algorithm(URM_test, rec)

Recommender performance is: MAP = 0.0317


In [29]:
rec_s_artist = BasicItemKNNRecommender(URM=URM_train, shrinkage=10.0, k=50)
rec_s_artist.fit(ICM_all_artist)
evaluate_algorithm(URM_test, rec_s_artist)

Normalized
Computed
Removed diagonal
Applied shrinkage
Converted to csr
Item 0 of 20635
Item 10000 of 20635
Item 20000 of 20635
Recommender performance is: MAP = 0.0317


<h3>Albums</h3>

In [31]:
rec_s_album = BasicItemKNNRecommender(URM=URM_train, shrinkage=10.0, k=50)
rec_s_album.fit(ICM_all_album)
evaluate_algorithm(URM_test, rec_s_album)

Normalized
Computed
Removed diagonal
Applied shrinkage
Converted to csr
Item 0 of 20635
Item 10000 of 20635
Item 20000 of 20635
Recommender performance is: MAP = 0.0499


<h3>Durations</h3>

In [32]:
rec_s_duration = BasicItemKNNRecommender(URM=URM_train, shrinkage=10.0, k=50)
rec_s_duration.fit(ICM_all_duration)
evaluate_algorithm(URM_test, rec_s_duration)

Normalized
Computed
Removed diagonal
Applied shrinkage
Converted to csr
Item 0 of 20635
Item 10000 of 20635
Item 20000 of 20635
Recommender performance is: MAP = 0.0001


<h2>Let's try to add collaborative filtering</h2>

In [53]:
import scipy
from scipy.sparse.linalg import (inv, spsolve)
from sklearn.preprocessing import MinMaxScaler

class collaborative_filtering(object):
    
    def nonzeros(self, m, row):
        for index in range(m.indptr[row], m.indptr[row+1]):
            yield m.indices[index], m.data[index]
            
    def least_squares_cg(self, Cui, X, Y, lambda_val, cg_steps=3):
        users, features = X.shape

        YtY = Y.T.dot(Y) + lambda_val * np.eye(features)

        for u in range(users):

            x = X[u]
            r = -YtY.dot(x)

            for i, confidence in self.nonzeros(Cui, u):
                r += (confidence - (confidence - 1) * Y[i].dot(x)) * Y[i]

            p = r.copy()
            rsold = r.dot(r)

            for it in range(cg_steps):
                Ap = YtY.dot(p)
                for i, confidence in self.nonzeros(Cui, u):
                    Ap += (confidence - 1) * Y[i].dot(p) * Y[i]

                alpha = rsold / p.dot(Ap)
                x += alpha * p
                r -= alpha * Ap

                rsnew = r.dot(r)
                p = r + (rsnew / rsold) * p
                rsold = rsnew

            X[u] = x
            
    def recommend(self, playlist_id, at=10):
        
        # Get all interactions by the user
        user_interactions = self.Cui[playlist_id,:].toarray()

        # We don't want to recommend items the user has consumed. So let's
        # set them all to 0 and the unknowns to 1.
        user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
        user_interactions[user_interactions > 1] = 0

        # This is where we calculate the recommendation by taking the 
        # dot-product of the user vectors with the item vectors.
        rec_vector = self.X_sparse[playlist_id,:].dot(self.Y_sparse.T).toarray()

        # Let's scale our scores between 0 and 1 to make it all easier to interpret.
        min_max = MinMaxScaler()
        rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
        recommend_vector = user_interactions*rec_vector_scaled

        # Get all the artist indices in order of recommendations (descending) and
        # select only the top "num_items" items. 
        ranking = np.argsort(recommend_vector)[::-1]

        ranking = self._filter_seen(playlist_id, ranking)

        return ranking[:at]
    
    def _filter_seen(self, playlist_id, ranking):
        user_profile = self.Cui[playlist_id]
        seen = user_profile.indices
        unseen_mask = np.in1d(ranking, seen, assume_unique=True, invert=True)
        return ranking[unseen_mask]

    def fit(self, Cui, features=10, iterations=20, lambda_val=0.1):
        user_size, item_size = Cui.shape

        X = np.random.rand(user_size, features) * 0.01
        Y = np.random.rand(item_size, features) * 0.01

        self.Cui, self.Ciu = Cui, Cui.T

        for iteration in range(iterations):
            print ('iteration %d of %d' % (iteration+1, iterations))
            self.least_squares_cg(self.Cui, X, Y, lambda_val)
            self.least_squares_cg(self.Ciu, Y, X, lambda_val)

        self.X_sparse = sps.csr_matrix(X)
        self.Y_sparse = sps.csr_matrix(Y)
        
        
        

    
    

In [54]:
alpha_val = 15
conf_data = (URM_train * alpha_val).astype('double')
rec_collab = collaborative_filtering()
rec_collab.fit(conf_data, features=10, iterations=20, lambda_val=0.1)

iteration 1 of 20
iteration 2 of 20
iteration 3 of 20
iteration 4 of 20
iteration 5 of 20
iteration 6 of 20
iteration 7 of 20
iteration 8 of 20
iteration 9 of 20
iteration 10 of 20
iteration 11 of 20
iteration 12 of 20
iteration 13 of 20
iteration 14 of 20
iteration 15 of 20
iteration 16 of 20
iteration 17 of 20
iteration 18 of 20
iteration 19 of 20
iteration 20 of 20


In [ ]:
evaluate_algorithm(URM_test, rec_collab)

<h3>it sucks</h3>


<h2>MF</h2>

In [1]:
from Recommender import Recommender
import subprocess
import os, sys
import time
import numpy as np


class MF_BPR_Cython(Recommender):


    def __init__(self, URM_train, positive_threshold=4, recompile_cython = False,
                 num_factors=10):


        super(MF_BPR_Cython, self).__init__()


        self.URM_train = URM_train
        self.n_users = URM_train.shape[0]
        self.n_items = URM_train.shape[1]
        self.normalize = False
        self.num_factors = num_factors
        self.positive_threshold = positive_threshold

        if recompile_cython:
            print("Compiling in Cython")
            self.runCompilationScript()
            print("Compilation Complete")



    def fit(self, epochs=30, logFile=None, URM_test=None, filterTopPop = False, filterCustomItems = np.array([], dtype=np.int), minRatingsPerUser=1,
            batch_size = 1000, validate_every_N_epochs = 1, start_validation_after_N_epochs = 0,
            learning_rate = 0.05, sgd_mode='sgd', user_reg = 0.0, positive_reg = 0.0, negative_reg = 0.0):


        self.eligibleUsers = []

        # Select only positive interactions
        URM_train_positive = self.URM_train.copy()

        URM_train_positive.data = URM_train_positive.data >= self.positive_threshold
        URM_train_positive.eliminate_zeros()


        for user_id in range(self.n_users):

            start_pos = URM_train_positive.indptr[user_id]
            end_pos = URM_train_positive.indptr[user_id+1]

            numUserInteractions = len(URM_train_positive.indices[start_pos:end_pos])

            if  numUserInteractions > 0 and numUserInteractions<self.n_items:
                self.eligibleUsers.append(user_id)

        # self.eligibleUsers contains the userID having at least one positive interaction and one item non observed
        self.eligibleUsers = np.array(self.eligibleUsers, dtype=np.int64)
        self.sgd_mode = sgd_mode




        # Import compiled module
        from MatrixFactorization.Cython.MF_BPR_Cython_Epoch import MF_BPR_Cython_Epoch


        self.cythonEpoch = MF_BPR_Cython_Epoch(URM_train_positive,
                                                 self.eligibleUsers,
                                                 num_factors = self.num_factors,
                                                 learning_rate=learning_rate,
                                                 batch_size=1,
                                                 sgd_mode = sgd_mode,
                                                 user_reg=user_reg,
                                                 positive_reg=positive_reg,
                                                 negative_reg=negative_reg)


        self.batch_size = batch_size
        self.learning_rate = learning_rate


        start_time_train = time.time()

        for currentEpoch in range(epochs):

            start_time_epoch = time.time()

            if currentEpoch > 0:
                if self.batch_size>0:
                    self.epochIteration()
                else:
                    print("No batch not available")


            if (URM_test is not None) and (currentEpoch % validate_every_N_epochs == 0) and \
                            currentEpoch >= start_validation_after_N_epochs:

                print("Evaluation begins")

                self.W = self.cythonEpoch.get_W()
                self.H = self.cythonEpoch.get_H()

                results_run = self.evaluateRecommendations(URM_test,
                                                           minRatingsPerUser=minRatingsPerUser)

                self.writeCurrentConfig(currentEpoch, results_run, logFile)

                print("Epoch {} of {} complete in {:.2f} minutes".format(currentEpoch, epochs,
                                                                     float(time.time() - start_time_epoch) / 60))


            # Fit with no validation
            else:
                print("Epoch {} of {} complete in {:.2f} minutes".format(currentEpoch, epochs,
                                                                         float(time.time() - start_time_epoch) / 60))

        # Ensure W and H are up to date
        self.W = self.cythonEpoch.get_W()
        self.H = self.cythonEpoch.get_H()

        print("Fit completed in {:.2f} minutes".format(float(time.time() - start_time_train) / 60))

        sys.stdout.flush()




    def runCompilationScript(self):

        # Run compile script setting the working directory to ensure the compiled file are contained in the
        # appropriate subfolder and not the project root

        compiledModuleSubfolder = "/MatrixFactorization/Cython"
        fileToCompile_list = ['MF_BPR_Cython_Epoch.pyx']

        for fileToCompile in fileToCompile_list:

            command = ['python',
                       'compileCython.py',
                       fileToCompile,
                       'build_ext',
                       '--inplace'
                       ]


            output = subprocess.check_output(' '.join(command), shell=True, cwd=os.getcwd() + compiledModuleSubfolder)

            try:

                command = ['cython',
                           fileToCompile,
                           '-a'
                           ]

                output = subprocess.check_output(' '.join(command), shell=True, cwd=os.getcwd() + compiledModuleSubfolder)

            except:
                pass


        print("Compiled module saved in subfolder: {}".format(compiledModuleSubfolder))

        # Command to run compilation script
        #python compileCython.py MF_BPR_Cython_Epoch.pyx build_ext --inplace

        # Command to generate html report
        #subprocess.call(["cython", "-a", "MF_BPR_Cython_Epoch.pyx"])


    def epochIteration(self):

        self.cythonEpoch.epochIteration_Cython()




    def writeCurrentConfig(self, currentEpoch, results_run, logFile):

        current_config = {'learn_rate': self.learning_rate,
                          'num_factors': self.num_factors,
                          'batch_size': 1,
                          'epoch': currentEpoch}

        print("Test case: {}\nResults {}\n".format(current_config, results_run))

        sys.stdout.flush()

        if (logFile != None):
            logFile.write("Test case: {}, Results {}\n".format(current_config, results_run))
            logFile.flush()




    def recommend(self, user_id, n=None, exclude_seen=True, filterTopPop = False, filterCustomItems = False):

        # compute the scores using the dot product
        user_profile = self.URM_train[user_id]

        scores_array = np.dot(self.W[user_id], self.H.T)


        if self.normalize:
            # normalization will keep the scores in the same range
            # of value of the ratings in dataset
            rated = user_profile.copy()
            rated.data = np.ones_like(rated.data)
            if self.sparse_weights:
                den = rated.dot(self.W_sparse).toarray().ravel()
            else:
                den = rated.dot(self.W).ravel()
            den[np.abs(den) < 1e-6] = 1.0  # to avoid NaNs
            scores_array /= den

        if exclude_seen:
            scores_array = self._filter_seen_on_scores(user_id, scores_array)

        if filterTopPop:
            scores_array = self._filter_TopPop_on_scores(scores_array)

        if filterCustomItems:
            scores_array = self._filterCustomItems_on_scores(scores_array)


        # rank items and mirror column to obtain a ranking in descending score
        #ranking = scores.argsort()
        #ranking = np.flip(ranking, axis=0)

        # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
        # - Partition the data to extract the set of relevant items
        # - Sort only the relevant items
        # - Get the original item index
        relevant_items_partition = (-scores_array).argpartition(n)[0:n]
        relevant_items_partition_sorting = np.argsort(-scores_array[relevant_items_partition])
        ranking = relevant_items_partition[relevant_items_partition_sorting]


        return ranking
    

ModuleNotFoundError: No module named 'Recommender'

In [ ]:
And the cython code for the update

#cython: boundscheck=False
#cython: wraparound=False
#cython: initializedcheck=False
#cython: language_level=3
#cython: nonecheck=False
#cython: cdivision=True
#cython: unpack_method_calls=True
#cython: overflowcheck=False

#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION

from Recommender_utils import check_matrix
import numpy as np
cimport numpy as np
import time
import sys

from libc.math cimport exp, sqrt
from libc.stdlib cimport rand, RAND_MAX


cdef struct BPR_sample:
    long user
    long pos_item
    long neg_item


cdef class MF_BPR_Cython_Epoch:

    cdef int n_users
    cdef int n_items, num_factors
    cdef int numPositiveIteractions

    cdef int useAdaGrad, rmsprop

    cdef float learning_rate, user_reg, positive_reg, negative_reg

    cdef int batch_size, sparse_weights

    cdef long[:] eligibleUsers
    cdef long numEligibleUsers

    cdef int[:] seenItemsSampledUser
    cdef int numSeenItemsSampledUser

    cdef int[:] URM_mask_indices, URM_mask_indptr

    cdef double[:,:] W, H


    def __init__(self, URM_mask, eligibleUsers, num_factors,
                 learning_rate = 0.05, user_reg = 0.0, positive_reg = 0.0, negative_reg = 0.0,
                 batch_size = 1, sgd_mode='sgd'):

        super(MF_BPR_Cython_Epoch, self).__init__()


        URM_mask = check_matrix(URM_mask, 'csr')

        self.numPositiveIteractions = int(URM_mask.nnz * 1)
        self.n_users = URM_mask.shape[0]
        self.n_items = URM_mask.shape[1]
        self.num_factors = num_factors

        self.URM_mask_indices = URM_mask.indices
        self.URM_mask_indptr = URM_mask.indptr

        # W and H cannot be initialized as zero, otherwise the gradient will always be zero
        self.W = np.random.random((self.n_users, self.num_factors))
        self.H = np.random.random((self.n_items, self.num_factors))



        if sgd_mode=='sgd':
            pass
        else:
            raise ValueError(
                "SGD_mode not valid. Acceptable values are: 'sgd'. Provided value was '{}'".format(
                    sgd_mode))



        self.learning_rate = learning_rate
        self.user_reg = user_reg
        self.positive_reg = positive_reg
        self.negative_reg = negative_reg


        if batch_size!=1:
            print("MiniBatch not implemented, reverting to default value 1")
        self.batch_size = 1

        self.eligibleUsers = eligibleUsers
        self.numEligibleUsers = len(eligibleUsers)


    # Using memoryview instead of the sparse matrix itself allows for much faster access
    cdef int[:] getSeenItems(self, long index):
        return self.URM_mask_indices[self.URM_mask_indptr[index]:self.URM_mask_indptr[index + 1]]



    def epochIteration_Cython(self):

        # Get number of available interactions
        cdef long totalNumberOfBatch = int(self.numPositiveIteractions / self.batch_size) + 1


        cdef BPR_sample sample
        cdef long u, i, j
        cdef long index, numCurrentBatch
        cdef double x_uij, sigmoid

        cdef int numSeenItems

        # Variables for AdaGrad and RMSprop
        cdef double [:] sgd_cache
        cdef double cacheUpdate
        cdef float gamma

        cdef double H_i, H_j, W_u

        #
        # if self.useAdaGrad:
        #     sgd_cache = np.zeros((self.n_items), dtype=float)
        #
        # elif self.rmsprop:
        #     sgd_cache = np.zeros((self.n_items), dtype=float)
        #     gamma = 0.001


        cdef long start_time_epoch = time.time()
        cdef long start_time_batch = time.time()

        for numCurrentBatch in range(totalNumberOfBatch):

            # Uniform user sampling with replacement
            sample = self.sampleBatch_Cython()

            u = sample.user
            i = sample.pos_item
            j = sample.neg_item

            x_uij = 0.0

            for index in range(self.num_factors):

                x_uij += self.W[u,index] * (self.H[i,index] - self.H[j,index])

            # Use gradient of log(sigm(-x_uij))
            sigmoid = 1 / (1 + exp(x_uij))


            #   OLD CODE, YOU MAY TRY TO USE IT
            #
            # if self.useAdaGrad:
            #     cacheUpdate = gradient ** 2
            #
            #     sgd_cache[i] += cacheUpdate
            #     sgd_cache[j] += cacheUpdate
            #
            #     gradient = gradient / (sqrt(sgd_cache[i]) + 1e-8)
            #
            # elif self.rmsprop:
            #     cacheUpdate = sgd_cache[i] * gamma + (1 - gamma) * gradient ** 2
            #
            #     sgd_cache[i] += cacheUpdate
            #     sgd_cache[j] += cacheUpdate
            #
            #     gradient = gradient / (sqrt(sgd_cache[i]) + 1e-8)


            for index in range(self.num_factors):

                # Copy original value to avoid messing up the updates
                H_i = self.H[i, index]
                H_j = self.H[j, index]
                W_u = self.W[u, index]

                self.W[u, index] += self.learning_rate * (sigmoid * ( H_i - H_j ) - self.user_reg * W_u)
                self.H[i, index] += self.learning_rate * (sigmoid * ( W_u ) - self.positive_reg * H_i)
                self.H[j, index] += self.learning_rate * (sigmoid * (-W_u ) - self.negative_reg * H_j)



            if((numCurrentBatch%5000000==0 and not numCurrentBatch==0) or numCurrentBatch==totalNumberOfBatch-1):
                print("Processed {} ( {:.2f}% ) in {:.2f} seconds. Sample per second: {:.0f}".format(
                    numCurrentBatch*self.batch_size,
                    100.0* float(numCurrentBatch*self.batch_size)/self.numPositiveIteractions,
                    time.time() - start_time_batch,
                    float(numCurrentBatch*self.batch_size + 1) / (time.time() - start_time_epoch)))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_batch = time.time()


    def get_W(self):

        return np.array(self.W)


    def get_H(self):
        return np.array(self.H)



    cdef BPR_sample sampleBatch_Cython(self):

        cdef BPR_sample sample = BPR_sample()
        cdef long index
        cdef int negItemSelected

        # Warning: rand() returns an integer

        index = rand() % self.numEligibleUsers

        sample.user = self.eligibleUsers[index]

        self.seenItemsSampledUser = self.getSeenItems(sample.user)
        self.numSeenItemsSampledUser = len(self.seenItemsSampledUser)

        index = rand() % self.numSeenItemsSampledUser

        sample.pos_item = self.seenItemsSampledUser[index]


        negItemSelected = False

        # It's faster to just try again then to build a mapping of the non-seen items
        # for every user
        while (not negItemSelected):
            sample.neg_item = rand() % self.n_items

            index = 0
            while index < self.numSeenItemsSampledUser and self.seenItemsSampledUser[index]!=sample.neg_item:
                index+=1

            if index == self.numSeenItemsSampledUser:
                negItemSelected = True

return sample

In [ ]:
from Movielens10MReader import Movielens10MReader

dataReader = Movielens10MReader()

URM_train = dataReader.get_URM_train()
URM_test = dataReader.get_URM_test()

recommender = MF_BPR_Cython(URM_train, recompile_cython=False, positive_threshold=4)

logFile = open("Result_log.txt", "a")

recommender.fit(epochs=5, validate_every_N_epochs=2, URM_test=URM_test,
                logFile=logFile, batch_size=1, sgd_mode='sgd', learning_rate=1e-4)

#results_run = recommender.evaluateRecommendations(URM_test, at=5)
#print(results_run)